In [1]:
RDATE = 20210131
RMONTH = 'JANUARY'

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

#/*Save status report islamic into xlsx format*/
FACILITIES_STATUS = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\01_Corporate_Financing\\03_Facilities_Status\\"+str(RDATE)+"\\status report islamic_"+str(RDATE)+".xlsx")

#/*Approved_Master_Borrower*/
Approved_Master_Borrower = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\01_Corporate_Financing\\03_Facilities_Status\\"+str(RDATE)+"\\CB Newly Approved Master File_"+str(RDATE)+".xlsx", sheet_name='Approved_Master_Borrower')

#/*Accepted*/
Accepted = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\01_Corporate_Financing\\03_Facilities_Status\\"+str(RDATE)+"\\CB Newly Approved Master File_"+str(RDATE)+".xlsx", sheet_name='Accepted')

#/*Emplaced*/
Emplaced = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\01_Corporate_Financing\\03_Facilities_Status\\"+str(RDATE)+"\\CB Newly Approved Master File_"+str(RDATE)+".xlsx", sheet_name='Emplaced')


In [56]:
FACILITIES_STATUS.columns = FACILITIES_STATUS.columns.str.replace(" ", "_").str.replace("\n", "_").str.replace(".", "_")
FACILITIES_STATUS.columns = FACILITIES_STATUS.columns.str.strip()
FACILITIES_STATUS.columns = FACILITIES_STATUS.columns.str.upper()

Approved_Master_Borrower.columns = Approved_Master_Borrower.columns.str.replace(" ", "_")
Approved_Master_Borrower.columns = Approved_Master_Borrower.columns.str.strip()
Approved_Master_Borrower.columns = Approved_Master_Borrower.columns.str.upper()

Accepted.columns = Accepted.columns.str.replace(" ", "_")
Accepted.columns = Accepted.columns.str.strip()
Accepted.columns = Accepted.columns.str.upper()

Emplaced.columns = Emplaced.columns.str.replace(" ", "_")
Emplaced.columns = Emplaced.columns.str.strip()
Emplaced.columns = Emplaced.columns.str.upper()
Emplaced1 = Emplaced.iloc[np.where(~Emplaced.YEAR.isna())]

In [8]:
#/*Facilities status*/
FACILITIES_STATUS.FINANCING_TYPE = FACILITIES_STATUS.FINANCING_TYPE.str.strip()
FACILITIES_STATUS.FINANCING_TYPE = FACILITIES_STATUS.FINANCING_TYPE.str.upper()
FACILITIES_STATUS.FINANCING_TYPE.fillna('NA', inplace=True)

FACILITIES_STATUS1 = FACILITIES_STATUS.iloc[np.where(FACILITIES_STATUS.FINANCING_TYPE=='ISLAMIC')]

FACILITIES_STATUS1['NOC'] = 1


In [23]:
#/*APPROVAL- CHANGED IN JULY 2020*/
FACILITIES_STATUS1.APPROVED = FACILITIES_STATUS1.APPROVED.astype(str)
FACILITIES_STATUS1.APPROVED.replace({'NaT': 'NA'}, inplace=True)
FACILITIES_STATUS1.APPROVED.fillna('NA', inplace=True)

FACILITIES_STATUS1.loc[FACILITIES_STATUS1.APPROVED.str.contains(str(RDATE)[:4]+'-'+str(RDATE)[4:6]),'FACILITIES_STATUS'] = '01. Newly Approved'
FACILITIES_STATUS1.loc[FACILITIES_STATUS1.APPROVED.str.contains(str(RDATE)[:4]+'-'+str(RDATE)[4:6]),'BAL_MIL'] = FACILITIES_STATUS1.TOTAL_FUNDED_APP


In [24]:
#/*JUST ADDED IN JULY 2019*/
FACILITIES_STATUS1.LO_ACCEPTED = FACILITIES_STATUS1.LO_ACCEPTED.astype(str)
FACILITIES_STATUS1.LO_ACCEPTED.replace({'NaT': 'NA'}, inplace=True)
FACILITIES_STATUS1.LO_ACCEPTED.fillna('NA', inplace=True)

FACILITIES_STATUS1.STATUS = FACILITIES_STATUS1.STATUS.str.strip()
FACILITIES_STATUS1.STATUS.fillna('NA', inplace=True)

FACILITIES_STATUS1.loc[(FACILITIES_STATUS1.LO_ACCEPTED!='NA')&(FACILITIES_STATUS1.STATUS!='UNMATERIALISED'),'FACILITIES_STATUS'] = '02. Accepted'
FACILITIES_STATUS1.loc[(FACILITIES_STATUS1.LO_ACCEPTED!='NA')&(FACILITIES_STATUS1.STATUS!='UNMATERIALISED'),'BAL_MIL'] = FACILITIES_STATUS1.TOTAL_FUNDED_APP


In [43]:
#/*JUST ADDED IN MAY 2018*/
FACILITIES_STATUS1.DATE_EMPLACED = FACILITIES_STATUS1.DATE_EMPLACED.astype(str)
FACILITIES_STATUS1.DATE_EMPLACED.replace({'nan': 'NA'}, inplace=True)
FACILITIES_STATUS1.DATE_EMPLACED.fillna('NA', inplace=True)

FACILITIES_STATUS1.loc[(FACILITIES_STATUS1.DATE_EMPLACED.str.contains(str(RDATE)[:4]+'-'+str(RDATE)[4:6]))&(FACILITIES_STATUS1.STATUS=='FULLY EMPLACED'),'FACILITIES_STATUS'] = '03. Emplaced'
FACILITIES_STATUS1.loc[(FACILITIES_STATUS1.DATE_EMPLACED.str.contains(str(RDATE)[:4]+'-'+str(RDATE)[4:6]))&(FACILITIES_STATUS1.STATUS=='FULLY EMPLACED'),'BAL_MIL'] = FACILITIES_STATUS1.TOTAL_FUNDED_EMP


In [44]:
FACILITIES_STATUS1.FACILITIES_STATUS.fillna('NA',inplace=True)
FACILITIES_STATUS1.BAL_MIL.fillna(0,inplace=True)
FACILITIES_STATUS1.CUST_NO.fillna(0, inplace=True)

FS001 = FACILITIES_STATUS1.groupby(['CUST_NO','FACILITIES_STATUS'])[['BAL_MIL']].sum().reset_index()

FS002 = FS001
FS002['NOC'] = 1

MIB_FACILITIES_STATUS = FS002.groupby(['FACILITIES_STATUS'])[['NOC','BAL_MIL']].sum().reset_index()

In [66]:
#/*Newly Approved*/
Approved_Master_Borrower.APPROVAL_DATE = Approved_Master_Borrower.APPROVAL_DATE.astype(str)

Approved_Master_Borrower.MONTH = Approved_Master_Borrower.MONTH.str.strip()
Approved_Master_Borrower.MONTH = Approved_Master_Borrower.MONTH.str.upper()

NA = Approved_Master_Borrower.iloc[np.where((Approved_Master_Borrower.YEAR==int(str(RDATE)[:4]))&\
                                            (Approved_Master_Borrower.MONTH==RMONTH)&\
                                            (Approved_Master_Borrower.APPROVAL_DATE.str.contains(str(RDATE)[:4]+'-'+str(RDATE)[4:6])))]

NA['NOC'] = 1

NA.RISK_CATEGORY = NA.RISK_CATEGORY.str.strip()
NA.RISK_CATEGORY = NA.RISK_CATEGORY.str.upper()

NA.loc[NA.RISK_CATEGORY=='VERY LOW RISK', 'RISK_CAT'] = '01. Very Low Risk'
NA.loc[NA.RISK_CATEGORY=='LOW RISK', 'RISK_CAT'] = '02. Low Risk'
NA.loc[NA.RISK_CATEGORY=='MODERATE', 'RISK_CAT'] = '03. Moderate Risk'
NA.loc[NA.RISK_CATEGORY=='MODERATE RISK', 'RISK_CAT'] = '03. Moderate Risk'
NA.loc[NA.RISK_CATEGORY=='HIGH RISK', 'RISK_CAT'] = '04. High Risk'
NA.loc[NA.RISK_CATEGORY=='SUB STANDARD', 'RISK_CAT'] = '05. Sub Standard'
NA.loc[NA.RISK_CATEGORY=='UNRATED', 'RISK_CAT'] = '06. Unrated'
NA.loc[NA.RISK_CATEGORY=='SOVEREIGN', 'RISK_CAT'] = '07. Sovereign'
NA.loc[NA.RISK_CATEGORY=='PSE', 'RISK_CAT'] = '08. PSE'
NA.loc[NA.RISK_CAT.isna(), 'RISK_CAT'] = NA.RISK_CATEGORY


NA.FRR_CATEGORY = NA.FRR_CATEGORY.str.strip()
NA.FRR_CATEGORY = NA.FRR_CATEGORY.str.upper()

NA.loc[NA.FRR_CATEGORY=='SECURED','COLLATERAL'] = '01. Secured'
NA.loc[NA.FRR_CATEGORY=='PARTIALLY SECURED','COLLATERAL'] = '02. Partially Secured'
NA.loc[NA.FRR_CATEGORY=='CLEAN','COLLATERAL'] = '03. Clean'

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
D:\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [72]:
NA.NATURE_OF_BUSINESS=NA.NATURE_OF_BUSINESS.str.strip()
NA.NATURE_OF_BUSINESS=NA.NATURE_OF_BUSINESS.str.upper()

NA.loc[NA.NATURE_OF_BUSINESS.isin(['01120','0113X','01140','0119XA','0119XB','01261A','01261B','01262A','01262B','01263','01269','01273','01279','01291','01299','0130X','0141X','0144X','0145X','0146XA','0146XB','0146XC','0149X','01500','016XX','0170X','0210X','0220X','0230X','0240X','031XXA','031XXB','031XXC','031XXD','032XXA','032XXB','032XXC', '1120','113X','1140','119XA','119XB','1261A','1261B','1262A','1262B','1263','1269','1273','1279','1291','1299','130X','141X','144X','145X','146XA','146XB','146XC','149X','1500','16XX','170X','210X','220X','230X','240X','31XXA','31XXB','31XXC','31XXD','32XXA','32XXB','32XXC']),'INDUSTRY'] = "01. AGRICULTURE, FORESTRY AND FISHING"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['05XXX','0610X','0620X','0710X','07210','07291','07299','08XXX','0910XA','0910XB','0910XC','0910XD','0910XE','0910XF','0910XG','09900', '5XXX','610X','620X','710X','7210','7291','7299','8XXX','910XA','910XB','910XC','910XD','910XE','910XF','910XG','9900']),'INDUSTRY'] = "02. MINING AND QUARRYING"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['1010X','1020X','1030X','10401A','10401B','10402A','10402B','10499','1050X','106XX','107XX','10800','110XXA','110XXB','12000','13XXX','14XXX','151XX','1520X','16XXX','1701X','181XX','18200','19XXXA','19XXXB','20XXX','21XXX','22XXXA','22XXXB','22XXXC','22XXXD','22XXXE','22XXXF','22XXXG','23XXXA','23XXXB','23XXXC','2410XA','2410XB','24201','24202','24209A','24209B','2431XA','2431XB','24320','251XX','25200','25910','25920','25930','2599X','26300','265XX','2670X','26999','27500','27999','28170','2819XA','2819XB','2819XC','2819XD','2819XE','28220','28299A','28299B','28299C','28299D','28299E','29XXXA','29XXXB','29XXXC','301XXA','301XXB','301XXC','301XXD','30200','30999A','30999B','30999C','3100X','32500','3290X','33XXXA','33XXXB','33XXXC','33XXXD','33XXXE','33XXXF','33XXXG','33XXXH']),'INDUSTRY'] = "03. MANUFACTURING"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['3510XA','3510XB','3510XC','3510XD','3510XE','3510XF','3510XG','3520XA','3520XB','3530X']),'INDUSTRY'] =  "04. ELECTRICITY, GAS, STEAM AND AIR CONDITIONING SUPPLY"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['360XX','3700X','38112','38199','382XXA','382XXB','382XXC','3830X','39000']),'INDUSTRY'] = "05. WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['41001A','41001B','41001C','41001D','41001E','41001F','41001G','41001H','41002A','41002B','41002C','41002D','41002E','41002F','41002G','41002H','41002I','41002J','41003','41009','4210XA','4210XB','4210XC','4210XD','4210XE','4210XF','4210XG','4220XA','4220XB','4220XC','4220XD','429XXA','429XXB','429XXC','431XX','432XX','4330X','4390X']),'INDUSTRY'] = "06. CONSTRUCTION"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['451XXA','451XXB','451XXC','451XXD','451XXE','4520X','4530XA','4530XB','4540X','462XXA','462XXB','462XXC','463XXA','463XXB','463XXC','463XXD','4641X','46496','46499','46510','46595','46599A','46599B','46599C','46999','4711X','4719X','472XXA','472XXB','472XXC','472XXD','478XX','479XX','40799A','40799B','40799C','40799D','40799E']),'INDUSTRY'] = "07. WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['4911X','4912X','4921X','4922X','49230','49300','5011X','50121','50122','5021X','5022X','51101A','51101B','51102','51103','51201','51202','51203','5210XA','5210XB','5221X','5222X','5223X','52241','52249','52291','52292','52299','53XXX']),'INDUSTRY'] = "08. TRANSPORTATION AND STORAGE"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['55101A','55101B','55101C','55101D','55104','55105','55108','55199','5520X','55900','56101','56102','56103','56104','56105','56106','56107','562XX','56302','56303','56399A','56399B']),'INDUSTRY'] = "09. ACCOMMODATION AND FOOD SERVICE ACTIVITIES"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['58110','5812X','5813X','5819X','582XX','59XXX','60XXX','6110X','6120X','6130X','6190X','620XX','631XX','6391X','63990']),'INDUSTRY'] = "10. INFORMATION AND COMMUNICATION"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['64110','64191','64192','64193','64194','64195','64199','64200','64301','64302','64303','64304','64309','64910','64921','64922','64923','64924','64925','64929','64991','64992','64993','64999','651XX','6520X','65301','65302','66111','66112','66113','66114','66119','66121','66122','66123','66124','66125','66129','66191','66192','66199','66211','66212','66221','66222','66223','66224','66290','66301','66302','66303']),'INDUSTRY'] = "11. FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['68101A','68101B','68101C','68102A','68102B','68102C','68102D','68102E','68103A','68103B','68104A','68104B','68104C','68104D','68104E','68104F','68104G','68109','68201','68202','68203','68209']),'INDUSTRY'] = "12. REAL ESTATE ACTIVITIES"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['69XXX','70100','70201','70202','70203','70209','7110X','71200','72102','72103','72105','72106','72199','7220X','73100','73200','74XXX','75000']),'INDUSTRY'] = "13. PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['7710X','772XX','77301','77307','77399','77400','78XXX','79XXX','80XXX','81XXX','821XX','82200','8230X','82910','8292XA','8292XB','8292XC','82990']),'INDUSTRY'] = "14. ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['841XX','842XX','8430X']),'INDUSTRY'] = "15. PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY SOCIAL SECURITY"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['8510X','852XX','8530X','854XX','8550X']),'INDUSTRY'] = "16. EDUCATION"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['861XX','8620X','8690X','87XXX','88XXX']),'INDUSTRY'] = "17. HUMAN HEALTH AND SOCIAL WORK ACTIVITIES"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['9000X','910XX','92000','93XXX']),'INDUSTRY'] = "18. ARTS, ENTERTAINMENT AND RECREATION"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['94XXXA','94XXXB','95XXX','960XX']),'INDUSTRY'] = "19. OTHER SERVICE ACTIVITIES"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['TXXXX']),'INDUSTRY'] = "20. ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS; UNDIFFERENTIATED GOODS AND SERVICES PRODUCING ACTIVITIES OF HOUSEHOLDS FOR OWN USE"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['99000']),'INDUSTRY'] = "21. ACTIVITIES OF EXTRATERRITORIAL ORGANISATIONS AND BODIES"
NA.loc[NA.NATURE_OF_BUSINESS.isin(['97000']),'INDUSTRY'] = "22. HOUSEHOLD"
NA.loc[NA.INDUSTRY.isna(),'INDUSTRY'] = "23. NOT POPULATED"


D:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
D:\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
D:\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [74]:
NA_RISKCAT = NA.groupby(['RISK_CAT'])[['NOC','TOTAL_LIMIT']].sum().reset_index()
NA_COLLATERAL = NA.groupby(['COLLATERAL'])[['NOC','TOTAL_LIMIT']].sum().reset_index()
NA_INDUSTRY = NA.groupby(['INDUSTRY'])[['NOC','TOTAL_LIMIT']].sum().reset_index()


In [80]:
#/*Newly Approved listing*/
NA001 = NA.sort_values(by=['TOTAL_LIMIT'],ascending=[False])

NALIST = NA001.head(5).rename(columns={'INDUSTRY':'BNM_SECTOR_CODE','GROUP':'GROUP1','APPROVAL_DATE':'APPROVAL_DATE1'})[['NAME','TOTAL_LIMIT','GROUP1','BNM_SECTOR_CODE','RISK_CATEGORY','FRR','RATING_FROM_CRRS','CUSTOMER_TYPE','APPROVAL_DATE1','APPROVAL_LEVEL']]

In [89]:
#/*Accepted listing*/

Accepted.MONTH = Accepted.MONTH.str.strip()
Accepted.MONTH = Accepted.MONTH.str.upper()

Accepted.TOTAL_LIMIT.fillna(0, inplace=True)
Accepted.NAME.fillna('NA', inplace=True)
Accepted.RATING_FROM_CRRS.fillna('NA', inplace=True)
Accepted.FRR.fillna('NA', inplace=True)
Accepted.APPROVAL_LEVEL.fillna('NA', inplace=True)

AP001 = Accepted.iloc[np.where((Accepted.YEAR==int(str(RDATE)[:4]))&(Accepted.MONTH==RMONTH))]

AP002 = AP001.groupby(['NAME','RATING_FROM_CRRS','FRR','APPROVAL_LEVEL'])[['TOTAL_LIMIT']].sum().reset_index().sort_values(by=['TOTAL_LIMIT'],ascending=[False])

APLIST = AP002.head(10)[['NAME','TOTAL_LIMIT','RATING_FROM_CRRS','FRR','APPROVAL_LEVEL']]

In [100]:
#/*Emplace listing*/

Emplaced.MONTH = Emplaced.MONTH.str.strip()
Emplaced.MONTH = Emplaced.MONTH.str.upper()

Emplaced.DATE_OF_EMPLACE = Emplaced.DATE_OF_EMPLACE.astype(str)
Emplaced.DATE_OF_EMPLACE.replace({'NaT': 'NA'}, inplace=True)
Emplaced.DATE_OF_EMPLACE.fillna('NA', inplace=True)

Emplaced.TOTAL_LIMIT.fillna(0, inplace=True)
Emplaced.NAME.fillna('NA', inplace=True)
Emplaced.RATING_FROM_CRRS.fillna('NA', inplace=True)
Emplaced.FRR.fillna('NA', inplace=True)
Emplaced.APPROVAL_LEVEL.fillna('NA', inplace=True)

EP001 = Emplaced.iloc[np.where((Emplaced.YEAR==int(str(RDATE)[:4]))&(Emplaced.MONTH==RMONTH)&\
                                            (Emplaced.DATE_OF_EMPLACE.str.contains(str(RDATE)[:4]+'-'+str(RDATE)[4:6])))]

EP002 = EP001.groupby(['NAME','RATING_FROM_CRRS','FRR','APPROVAL_LEVEL'])[['TOTAL_LIMIT']].sum().reset_index().sort_values(by=['TOTAL_LIMIT'],ascending=[False])

EPLIST = EP002.head(10)[['NAME','TOTAL_LIMIT','RATING_FROM_CRRS','FRR','APPROVAL_LEVEL']]

In [109]:
writer2 = pd.ExcelWriter(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\01_Corporate_Financing\04_Dataset\\"+str(RDATE)+"\\MIB_CB_BNM_SUMM_"+str(RDATE)+"(python).xlsx",engine='xlsxwriter')

MIB_FACILITIES_STATUS.to_excel(writer2, sheet_name='FACILITIES STATUS', index = False)
NA_RISKCAT.to_excel(writer2, sheet_name='NA RISK CATEGORY', index = False)
NA_COLLATERAL.to_excel(writer2, sheet_name='NA COLLATERAL', index = False)

NA_INDUSTRY.to_excel(writer2, sheet_name='NA ECONOMIC SECTOR', index = False)
NALIST.to_excel(writer2, sheet_name='Newly Approved', index = False)
APLIST.to_excel(writer2, sheet_name='Accepted&Pending Emplacement', index = False)
EPLIST.to_excel(writer2, sheet_name='Emplaced', index = False)

writer2.save()